<a href="https://colab.research.google.com/github/andohyung/MS_Project/blob/main/FineTuning_BART.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


환경 설정

In [2]:
pip install transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


데이터셋 로드

In [ ]:
from datasets import load_dataset

# 데이터셋 로드
dataset = load_dataset("armanc/scientific_papers",name="pubmed", split="train")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.27k [00:00<?, ?B/s]

scientific_papers.py:   0%|          | 0.00/5.35k [00:00<?, ?B/s]

The repository for armanc/scientific_papers contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/armanc/scientific_papers.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/119924 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6633 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6658 [00:00<?, ? examples/s]

모델과 토크나이저 설정

In [7]:
from transformers import BartTokenizer, BartForConditionalGeneration

# BART 토크나이저와 모델 로드
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

데이터 전처리

In [6]:
def preprocess_function(examples):
    inputs = examples["article"]  # 논문의 본문
    targets = examples["abstract"]  # 논문의 요약
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=256, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# 데이터셋 전처리
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# 디렉토리 경로 설정 (예: "/content/drive/MyDrive/processed_datasets")
save_path = "/content/drive/MyDrive/멋사_실전프로젝트1/tokenized_datasets"

# 데이터셋을 해당 경로에 저장
tokenized_datasets.save_to_disk(save_path)



Saving the dataset (0/7 shards):   0%|          | 0/119924 [00:00<?, ? examples/s]

In [4]:
from datasets import load_from_disk

# 저장된 데이터셋 로드
loaded_datasets = load_from_disk('/content/drive/MyDrive/멋사_실전프로젝트1/tokenized_datasets')

데이터 로더 설정

In [5]:
from torch.utils.data import DataLoader
from transformers import default_data_collator

# DataLoader 설정 (collate_fn 추가)
train_dataloader = DataLoader(
    loaded_datasets,
    batch_size=32,
    shuffle=True,
    collate_fn=default_data_collator  # Tensor 변환을 자동 처리
)

In [1]:
import torch
torch.cuda.empty_cache()

모델 학습 설정 및 FINE-TUNING

In [ ]:
from transformers import AdamW, get_scheduler
import torch

# Optimizer와 스케줄러 설정
optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 1
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

# 모델 학습 준비
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# 모델 학습
model.train()
for epoch in range(num_epochs):  # epoch 루프 추가
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    print(f"Epoch {epoch + 1} completed. Loss: {loss.item()}")

# 모델과 토크나이저 저장 (fine-tuning 완료 후)
model.save_pretrained("/content/drive/MyDrive/멋사_실전프로젝트1/fine_tuned_model")
tokenizer.save_pretrained("/content/drive/MyDrive/멋사_실전프로젝트1/fine_tuned_model")

파인튜닝 된 모델 및 학습 데이터로 지표 평가

In [ ]:
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import BartTokenizer, default_data_collator

# 토크나이저 로드
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

# validation 데이터셋 로드 및 전처리
dataset = load_dataset("armanc/scientific_papers", name="pubmed", split="validation")

# 전처리 함수
def preprocess_function(examples):
    inputs = examples["article"]  # 요약할 본문
    targets = examples["abstract"]  # 참조 요약 (정답)
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=256, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# 전처리 적용
tokenized_validation = dataset.map(preprocess_function, batched=True)

# validation DataLoader 설정
validation_dataloader = DataLoader(
    tokenized_validation,
    batch_size=8,
    collate_fn=default_data_collator
)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


파인튜닝된 모델로 성능 지표 구하기

In [11]:
#pip install -U evaluate
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=d03fe9126d7d6c5deb170fd69332420225a5c7f123674e942020a8e1fe5f3a66
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [9]:
                                                                                                                                                                 from transformers import BartForConditionalGeneration, BartTokenizer

# 모델과 토크나이저 로드
model = BartForConditionalGeneration.from_pretrained("/content/drive/MyDrive/멋사_실전프로젝트1/fine_tuned_model")
tokenizer = BartTokenizer.from_pretrained("/content/drive/MyDrive/멋사_실전프로젝트1/fine_tuned_model")


In [ ]:
from evaluate import load
import torch

# 모델을 GPU로 이동
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# ROUGE 평가지표 로드
rouge = load("rouge")

# fine-tuning된 모델로 평가 수행
model.eval()
with torch.no_grad():
    for batch in validation_dataloader:
        # batch의 모든 텐서를 GPU로 이동
        batch = {k: v.to(device) for k, v in batch.items()}

        # 모델 생성
        outputs = model.generate(batch["input_ids"], max_length=256, num_beams=4, early_stopping=True)

        # 예측 및 참조 텍스트 디코딩
        predictions = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        references = tokenizer.batch_decode(batch["labels"], skip_special_tokens=True)

        # ROUGE 평가
        rouge.add_batch(predictions=predictions, references=references)

# ROUGE 결과 출력
result = rouge.compute()
print(result)


{'rouge1': 0.4233794125909605, 'rouge2': 0.17460346352492725, 'rougeL': 0.2568563807251305, 'rougeLsum': 0.3723640190726435}


파인튜닝 된 모델에 정답 직접 입력을 통한 지표 확인

In [12]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration
from rouge_score import rouge_scorer

# 파일 경로에 있는 파인튜닝된 모델 로드
# model_path = "/content/drive/MyDrive/fine_tuned_model"  # 파인튜닝된 모델의 디렉토리 경로
# tokenizer = BartTokenizer.from_pretrained(model_path)
# model = BartForConditionalGeneration.from_pretrained(model_path)

# 테스트할 논문 본문과 정답 요약 (예시)
test_article = """
We introduce a new language representa-tion model called BERT,
which stands forBidirectional Encoder Representations fromTransformers.
BERT is designed to pre-train deep bidirectional representations fromunlabeled text by jointly conditioning on both left and right context in all layers.
It obtains new state-of-the-art re-sults on eleven natural language processing tasks.
"""
reference_summary = """
BERT (Bidirectional Encoder Representations from Transformers) is a language model that learns deep bidirectional representations by considering both left and right context.
It uses Masked Language Modeling and Next Sentence Prediction to improve understanding across tasks.
BERT achieves state-of-the-art results on multiple NLP benchmarks with minimal task-specific adjustments, outperforming prior unidirectional models like GPT.
Its open-source release has made it a cornerstone in NLP research.
"""

# 입력 데이터 토크나이징
inputs = tokenizer(test_article, max_length=1024, truncation=True, return_tensors="pt")

# 모델 요약 생성
summary_ids = model.generate(inputs["input_ids"], max_length=150, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)
generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# 정답 요약과 모델 요약 출력
print("Generated Summary:", generated_summary)
print("Reference Summary:", reference_summary)

# ROUGE 스코어 계산
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score(reference_summary, generated_summary)

# ROUGE 스코어 출력
print("\nROUGE Scores:")
print("ROUGE-1:", scores['rouge1'])
print("ROUGE-2:", scores['rouge2'])
print("ROUGE-L:", scores['rougeL'])


Generated Summary:  we introduce a new language representa-tion model called bERT, 
 which stands for bidirectional Encoder Representations fromTransformers. 
 it is designed to pre - train deep bidirectionally representations fromunlabeled text by jointly conditioning on both left and right context in all layers. it obtains new state-of-the - art re-sults on eleven natural language processing tasks. 
Reference Summary: 
BERT (Bidirectional Encoder Representations from Transformers) is a language model that learns deep bidirectional representations by considering both left and right context. 
It uses Masked Language Modeling and Next Sentence Prediction to improve understanding across tasks. 
BERT achieves state-of-the-art results on multiple NLP benchmarks with minimal task-specific adjustments, outperforming prior unidirectional models like GPT. 
Its open-source release has made it a cornerstone in NLP research.


ROUGE Scores:
ROUGE-1: Score(precision=0.509090909090909, recall=0.4, 